# Попытка описать метод калибровки камер по спектрофотометрическому каталогу

В спектрофотометрическом каталоге **Харитонова** приведены внеатмосферные квазимонохроматические освещенности $E(\lambda)$, создаваемые интегральными спектрами некоторых звезд (567 штук).  
Причем освещенности рассчитаны на $\Delta\lambda=1 см$, т.е. выражены в $[\frac{эрг}{см^2 \cdot с \cdot см}]$.

## В качестве примера возьмем Вегу ($\alpha Lyr$)
У меня есть может быть глупый вопрос о том, можно ли использовать для калибровки Полярную звезду? У нее же переменный блеск (относительные вариации хаотичные $\leq5\%$ от звездной величины, с периодом 4 дня).  

**Вега**: для линии $\lambda=6300 нм$ средняя освещенность будет (по каталогу) $E_{\lambda}=\frac{(239+234)\cdot 10 ^{-3}}{2}=236.5\cdot10^{-3} \frac{эрг}{см^2 \cdot с \cdot см}$.  
    Эту величину нужно пересчитать в поверхностную яркость (будем считать, что мы уже знаем, что звезда на кадре занимает 6 пикселей).

Все последующие формулы и обозначения взяты из документа, подготовленного В.В. Клименко "О калибровке ПЗС.doc".   
Пересчитаем освещенность в поток фотонов: $$F_{\lambda}[\frac{фотон}{с \cdot см^2 \cdot см}] = \frac {\lambda} {h\cdot c} E_{\lambda} [\frac{эрг}{с \cdot см^2 \cdot см}],$$
где $h =6.626·10^{-27} эрг\cdot с$ – постоянная Планка, $c = 3·10^{10} см/с$ – скорость света, $ \lambda = 6300А=6.3\cdot 10 ^{-5} см$.
Поверхностная яркость расфокусированного изображения звезды в Рэлеях: $$ B^{экв}[Рэлей] = \frac{4\pi}{10^6}\cdot \frac{F_{\lambda} \Delta\lambda_{eff}}{\Omega_m},$$
где $\Delta\lambda_{eff}$ - полоса фильтра (для KEO - $\Delta\lambda_{eff}=20A=20\cdot 10^{-8} см$), $\Omega_m$ - телесный угол, соответствующий расфокусированному изображению звезды.  
Пусть звезда занимает $n$ пикселей, тогда $\Omega_m=n \cdot \Omega_m^{pix}$, где $\Omega_m^{pix}$ - телесный угол одного пикселя.

$\Omega_m^{pix}$ вообще говоря является функцией зенитного угла, т.е. $\Omega_m^{pix}=\Omega_m^{pix}(\theta)$. В документе **"solid_angles_det.ipynb"** выведена расчетная формула: $$\Omega_m^{pix} (\theta)= (\rho_{\theta} \cdot \frac {180} {\pi})^{-2}\cdot \frac{\sin \theta}{\theta} ,$$  
где $\rho_{\theta}$ - количество пикселей, приходящихся на 1 градус зенитного угла. Для зенитно-эквидистантной проекции, используемой в камере KEO, $\rho_{\theta}$ - постоянная величина. Для кадра 511*511 (бинирование 4), $\rho_{\theta}\approx 3.575 \frac {пиксель}{^\circ}$.  
По этой формуле можно посчитать телесный угол пикселя в центре кадра, в области наблюдения свечения и на краю кадра:  
$ \Omega_m^{pix} = 23.8342185715 \cdot 10^{-6} стерад $ - в центре кадра;  
$ \Omega_m^{pix} = 22.2967446323 \cdot 10^{-6} стерад $ - в области свечения ($ \theta \approx 36^{\circ}$);  
$ \Omega_m^{pix}  = 18.0384453267 \cdot 10^{-6} стерад $ - на краю кадра ($ \theta \approx 72^{\circ}$).  

Полагаю разумным в дальнейшем использовать величину $ \Omega_m^{pix} = 22.3 \cdot 10^{-6} стерад $.

Если собрать всё вместе, то получим расчетную формулу:  
$$ B^{экв}[Рэлей] = \frac{4\pi}{10^6} \cdot \frac{\lambda\Delta\lambda_{eff}}{h c \Omega_m^{pix}} \cdot \frac {E_\lambda} {n},$$
в которой $\lambda$ и $\Delta\lambda$ нужно брать в см, $h$ в $эрг \cdot с$, $c$ в $\frac{см}{с}$, $E_{\lambda}$ - как в каталоге (в $\frac{эрг}{с \cdot см^2 \cdot см}$ ).  
Если поставить все константы, то получим: $$ B^{экв}[Рэлей] = 3.572 \cdot 10^4 \frac{1}{n} E_{\lambda} [\frac{эрг}{с \cdot см^2 \cdot см}]$$  
Еще раз напишу здесь, что полученная формула вообще говоря справедлива только в области зенитных углов до $40^{\circ}$. К центру кадра коэффициент (3.572) будет меньше (но не более чем на 5%), а к краю кадра до 20% больше.

In [13]:
4*pi*6.3*20/6.626/3/22.3

3.5719293461623876

Для **Веги**, занимающей 6 пикселей, получим: 

In [15]:
(3.572*10**4)*(236.5*10**-3)/6

1407.9633333333334

$$ B^{экв} \approx 1408 Рэлей$$

По снимку (файл "20140824_2408.fit", разрешение 511*511, бинирование 4, выдержка 30 с, после вычитания master_dark кадра) определим, что:  
1. Вега занимает 6 пикселей.
2. Среднее значение в единицах АЦП каждого из этих 6 пикселей после вычитания фона - $4374$.  

In [16]:
1408/4374

0.3219021490626429

Таким образом, коэффициент пропорциональности $R=\frac{1408}{4374}\approx 0.32 \frac{Рэлей}{ед.АЦП}$

### Замечания к методу:
1. Из каталога Харитонова я выбирал для калибровки только звезды, видимые в интервале зенитных углов (45,60) градусов, занимающие площадь больше 3 пикселей (чтобы приблизиться к требованию расфокусировки).
2. Для разных звезд коэффициент пропорциональности варьировался в интервале (0.2 до 0.5), но если брать медианное значение ~ по 5 звездам, то получался стабильный результат для всех кадров (примерно 0.35 Рэлей на ед.АЦП, в том числе и для измерений, проведенных 26.08.14).
3. В принципе в расчетную формулу можно ввести зависимость телесного угла пикселя от зенитного угла. Может в этом случае вариации коэффициента $R$ от звезды к звезде будут меньше (но не более чем на 5%). 
4. Для других выдержек и другого разрешения, коэффициент 3.572 надо пересчитать.